<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=7219cd9d54b024a0d49ce4cbefbe0c0968c7cb133507a83dd5af96c3fba21d1d
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.4
    Uninstalling protobuf-6.33.4:
      Successfully uninstalled protobuf-6.33.4


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-19 13:10:45
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.39 C
-------------------
Today PnL: -92.95 K (-0.66%)
Current PnL: -29.98 L (-19.12%)
CY Booked + Current PnL: -15.69 L (-10.0%)
-------------------
Total profit:  1.64 L
Total loss:  -31.62 L
-------------------
Total Booked + Current PnL: 11.29 L (8.7%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.27%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.34 C
Est FTT PnL: 95.33 L (68.53%)
Deployed:  1.30 C
Current:  1.39 C
CAGR/XIRR %: 3.69%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,-0.35,15.60,5.01,21.40,15373.0,41418.0,306844.0,7.22,60.0,X-LC,6.79,14.0,2.69,2.21,22.92,XY25,NTT,FMCG
17,COALINDIA,484.83,-0.17,5.80,12.69,19.23,20201.0,8732.0,159192.0,24.27,67.0,L-LC,10.45,182.0,0.43,1.14,28.31,XY25,ATH,MINING
77,TTKPRESTIG,770.00,-2.80,-24.57,32.69,0.09,24851.0,-24758.0,76019.0,76.52,29.0,M-SC,4.46,253.0,-1.00,0.55,0.00,OX40N,NTT,DURABLES
49,MASFIN,398.61,-0.05,-4.72,28.09,22.05,26225.0,-4620.0,93360.0,-17.80,45.0,H-SC,5.16,168.0,-0.18,0.67,35.71,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-5.38,-16.89,20.35,0.02,29521.0,-29491.0,145068.0,120.98,46.0,M-SC,9.33,240.0,-1.00,1.04,26.18,OX40N,NTT,PAINTS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,JSWINFRA,342.0,7.18,-5.97,23.82,16.43,49672.0,-13243.0,208531.0,-20.37,52.0,X-MC,7.50,66.0,-0.27,1.50,23.07,X40N,NTT,REALTY
33,HINDUNILVR,2922.0,1.90,-6.44,21.49,13.67,51688.0,-16543.0,240520.0,-13.44,59.0,X-LC,2.58,9.0,-0.32,1.73,13.55,XY25,NTT,FMCG
56,RAJOOENG,143.1,1.77,-35.51,116.49,39.61,77000.0,-36400.0,66100.0,-56.41,46.0,H-SC,30.22,136.0,-0.47,0.48,6.99,AR,ATH,MISC
62,SATIN,274.0,1.45,-15.23,79.11,51.83,185285.0,-42072.0,234212.0,-23.78,68.0,H-SC,3.82,148.0,-0.23,1.68,14.10,XY24,NTT,FINANCE
0,5PAISA,593.0,1.41,-27.65,56.05,12.90,88604.0,-60416.0,158080.0,133.17,61.0,H-SC,8.58,173.0,-0.68,1.14,31.44,OX40N,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-7.55,1.41,69.87,72.27,120064.0,2391.0,171839.0,-12.89,36.0,M-LC,5.75,101.0,0.02,1.24,7.67,XR,NTT,IT
39,INDIGOPNTS,1408.00,-5.38,-16.89,20.35,0.02,29521.0,-29491.0,145068.0,120.98,46.0,M-SC,9.33,240.0,-1.00,1.04,26.18,OX40N,NTT,PAINTS
59,RELIANCE,1952.00,-3.52,-9.71,38.77,25.30,70349.0,-19512.0,181451.0,-12.63,25.0,X-LC,6.55,26.0,-0.28,1.30,21.54,XY25,BTT,REFINERIES
38,INDIAMART,4810.62,-3.49,-5.07,117.76,106.72,137876.0,-6254.0,117082.0,-52.92,50.0,H-SC,8.36,138.0,-0.05,0.84,17.12,AR,ATH,MISC
55,RAJESHEXPO,518.00,-3.16,-67.57,208.61,0.07,92790.0,-92697.0,44480.0,1530.05,39.0,L-SC,1.24,269.0,-1.00,0.32,10.62,OX40N,NTT,JEWELLERY


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.0,-0.42,0.01,20.72,20.74,42103.0,28.0,203198.0,31.15,55.0,M-MC,5.13,189.0,0.0,1.46,10.63,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-2.61,-3.80,116.12,107.91,164932.0,-5608.0,142036.0,-21.77,47.0,M-SC,9.97,220.0,-0.03,1.02,5.90,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-5.38,-16.89,20.35,0.02,29521.0,-29491.0,145068.0,120.98,46.0,M-SC,9.33,240.0,-1.00,1.04,26.18,OX40N,NTT,PAINTS
72,TATAELXSI,9161.00,-0.74,-19.81,64.65,32.03,68345.0,-26119.0,105716.0,-13.18,55.0,H-SC,6.41,157.0,-0.38,0.76,18.41,OX40N,NTT,IT
47,KANSAINER,340.00,-0.52,-23.17,47.68,13.47,98792.0,-62469.0,207198.0,-68.51,45.0,H-SC,4.10,159.0,-0.63,1.49,5.35,XY24,NTT,PAINTS
66,SIS,528.00,0.32,-23.96,59.47,21.26,50211.0,-26602.0,84430.0,1992.23,48.0,H-SC,4.76,163.0,-0.53,0.61,14.07,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.00,-0.42,0.01,20.72,20.74,42103.0,28.0,203198.0,31.15,55.0,M-MC,5.13,189.0,0.00,1.46,10.63,XY25,NTT,FINANCE
17,COALINDIA,484.83,-0.17,5.80,12.69,19.23,20201.0,8732.0,159192.0,24.27,67.0,L-LC,10.45,182.0,0.43,1.14,28.31,XY25,ATH,MINING
84,WIPRO,420.00,-7.55,1.41,69.87,72.27,120064.0,2391.0,171839.0,-12.89,36.0,M-LC,5.75,101.0,0.02,1.24,7.67,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-7.55,1.41,69.87,72.27,120064.0,2391.0,171839.0,-12.89,36.0,M-LC,5.75,101.0,0.02,1.24,7.67,XR,NTT,IT
25,FINCABLES,1641.55,-2.61,-3.80,116.12,107.91,164932.0,-5608.0,142036.0,-21.77,47.0,M-SC,9.97,220.0,-0.03,1.02,5.90,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-3.49,-5.07,117.76,106.72,137876.0,-6254.0,117082.0,-52.92,50.0,H-SC,8.36,138.0,-0.05,0.84,17.12,AR,ATH,MISC
37,IEX,219.00,-1.77,-7.14,60.05,48.63,111500.0,-14276.0,185678.0,-38.14,45.0,H-SC,13.42,137.0,-0.13,1.34,3.65,XR,NTT,MISC
49,MASFIN,398.61,-0.05,-4.72,28.09,22.05,26225.0,-4620.0,93360.0,-17.80,45.0,H-SC,5.16,168.0,-0.18,0.67,35.71,XR,ATH,FINANCE


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.0,-0.82,-31.71,130.20,57.19,289954.0,-103428.0,222699.0,-67.71,16.0,X-MC,11.76,58.0,-0.36,1.60,0.00,XY24,NTT,FMCG
43,ITC,452.0,0.90,-18.59,36.08,10.78,70585.0,-44682.0,195636.0,-51.37,17.0,X-LC,10.81,1.0,-0.63,1.41,0.90,X40,NTT,FMCG
11,BATAINDIA,2096.0,-1.12,-42.42,134.81,35.20,99881.0,-54580.0,74090.0,-2.82,22.0,X-SC,17.79,93.0,-0.55,0.53,0.00,X40,NTT,FOOTWEAR
58,RELAXO,1176.0,-3.14,-51.29,212.68,52.31,150380.0,-74453.0,70707.0,-48.12,23.0,X-SC,8.47,92.0,-0.50,0.51,0.00,X40N,NTT,FOOTWEAR
59,RELIANCE,1952.0,-3.52,-9.71,38.77,25.30,70349.0,-19512.0,181451.0,-12.63,25.0,X-LC,6.55,26.0,-0.28,1.30,21.54,XY25,BTT,REFINERIES


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4311.59,-1.47,-13.13,36.46,18.55,119803.0,-49649.0,328588.0,-25.24,49.0,X-LC,1.41,3.0,-0.41,2.36,11.79,X40,ATH,IT
45,JIOFIN,387.00,-1.29,-11.48,40.63,24.48,96160.0,-30693.0,236672.0,-12.95,31.0,X-LC,1.85,37.0,-0.32,1.70,37.18,XY24,BTT,FINANCE
78,UNITDSPR,1644.00,-1.09,-2.01,23.24,20.77,53324.0,-4696.0,229448.0,-10.48,36.0,X-MC,2.05,70.0,-0.09,1.65,4.77,X40N,NTT,BREWERIES
20,DABUR,735.00,-0.15,0.45,43.05,43.70,107278.0,1125.0,249193.0,-7.78,54.0,X-MC,2.12,73.0,0.01,1.79,15.57,XY24,BTT,FMCG
35,ICICIGI,2252.93,-0.38,-1.31,20.92,19.34,41315.0,-2620.0,197489.0,-22.15,34.0,X-MC,2.56,71.0,-0.06,1.42,14.42,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-0.82,-31.71,130.20,57.19,289954.0,-103428.0,222699.0,-67.71,16.0,X-MC,11.76,58.0,-0.36,1.60,0.0,XY24,NTT,FMCG
8,BAJAJHFL,181.50,-0.96,-21.62,98.36,55.47,187559.0,-52600.0,190686.0,-30.61,28.0,X-MC,16.09,63.0,-0.28,1.37,0.0,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-1.12,-42.42,134.81,35.20,99881.0,-54580.0,74090.0,-2.82,22.0,X-SC,17.79,93.0,-0.55,0.53,0.0,X40,NTT,FOOTWEAR
53,PGHH,17907.65,-0.42,-8.62,46.86,34.19,91426.0,-18416.0,195104.0,-35.73,26.0,X-MC,4.88,60.0,-0.20,1.40,0.0,X40,ATH,FMCG
58,RELAXO,1176.00,-3.14,-51.29,212.68,52.31,150380.0,-74453.0,70707.0,-48.12,23.0,X-SC,8.47,92.0,-0.50,0.51,0.0,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-0.85,-31.19,107.60,42.84,48128.0,-20277.0,44729.0,-54.82,39.0,X-SC,23.79,83.0,-0.42,0.32,1.19,XY24,NTT,MISC
58,RELAXO,1176.00,-3.14,-51.29,212.68,52.31,150380.0,-74453.0,70707.0,-48.12,23.0,X-SC,8.47,92.0,-0.50,0.51,0.00,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.12,-42.42,134.81,35.20,99881.0,-54580.0,74090.0,-2.82,22.0,X-SC,17.79,93.0,-0.55,0.53,0.00,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,-1.28,-5.69,31.56,24.08,37786.0,-7223.0,119727.0,-13.35,38.0,X-SC,6.01,91.0,-0.19,0.86,12.81,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.52,-17.01,72.71,43.32,98275.0,-27712.0,135160.0,-28.68,49.0,X-SC,8.11,90.0,-0.28,0.97,3.78,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,0.90,-18.59,36.08,10.78,70585.0,-44682.0,195636.0,-51.37,17.0,X-LC,10.81,1.0,-0.63,1.41,0.90,X40,NTT,FMCG
75,TMPV,600.00,-2.66,-29.45,74.32,22.98,118440.0,-66533.0,159365.0,-26.74,35.0,X-LC,3.41,2.0,-0.56,1.15,0.23,XY24,NTT,AUTO
73,TCS,4311.59,-1.47,-13.13,36.46,18.55,119803.0,-49649.0,328588.0,-25.24,49.0,X-LC,1.41,3.0,-0.41,2.36,11.79,X40,ATH,IT
81,VBL,671.64,-0.45,0.63,34.83,35.67,110697.0,1978.0,317820.0,-11.39,57.0,X-LC,2.97,5.0,0.02,2.29,14.57,X40N,ATH,FMCG
41,INFY,1972.00,-0.73,14.20,17.56,34.24,62743.0,44420.0,357308.0,-10.98,63.0,X-LC,7.47,6.0,0.71,2.57,23.65,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.68,-30.06,89.99,32.89,48596.0,-23204.0,54002.0,-767.12,71.0,L-MC,9.58,259.0,-0.48,0.39,48.72,XR,NTT,BANKS
13,BSOFT,831.70,-1.59,-22.73,95.14,50.79,103399.0,-31972.0,108681.0,-1.04,50.0,H-SC,9.80,171.0,-0.31,0.78,26.54,XR,ATH,IT
17,COALINDIA,484.83,-0.17,5.80,12.69,19.23,20201.0,8732.0,159192.0,24.27,67.0,L-LC,10.45,182.0,0.43,1.14,28.31,XY25,ATH,MINING
36,ICICIPRULI,790.00,-2.39,9.51,19.37,30.73,41790.0,18742.0,215747.0,-16.10,50.0,X-MC,7.53,79.0,0.45,1.55,23.50,X40,ATH,INSURANCE
5,ASIANPAINT,3460.26,0.15,-3.50,25.33,20.95,61544.0,-8800.0,242968.0,-4.43,45.0,X-LC,12.81,36.0,-0.14,1.75,30.90,X40,ATH,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,1.41,-27.65,56.05,12.90,88604.0,-60416.0,158080.0,133.17,61.0,H-SC,8.58,173.0,-0.68,1.14,31.44,OX40N,NTT,FINANCE
40,INDUSINDBK,1800.00,-0.68,-30.06,89.99,32.89,48596.0,-23204.0,54002.0,-767.12,71.0,L-MC,9.58,259.0,-0.48,0.39,48.72,XR,NTT,BANKS
17,COALINDIA,484.83,-0.17,5.80,12.69,19.23,20201.0,8732.0,159192.0,24.27,67.0,L-LC,10.45,182.0,0.43,1.14,28.31,XY25,ATH,MINING
62,SATIN,274.00,1.45,-15.23,79.11,51.83,185285.0,-42072.0,234212.0,-23.78,68.0,H-SC,3.82,148.0,-0.23,1.68,14.10,XY24,NTT,FINANCE
57,RECLTD,446.00,-0.42,0.01,20.72,20.74,42103.0,28.0,203198.0,31.15,55.0,M-MC,5.13,189.0,0.00,1.46,10.63,XY25,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.62
1,25,44.80
2,50,76.71


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.61
MC    30.66
LC    26.76
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.22
X40      24.17
X40N     12.87
XY25     10.88
XR        8.86
AR        8.40
OX40N     7.69
SR        0.94
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.28
X-LC    23.32
H-SC    22.81
M-SC    11.12
X-SC     7.90
H-MC     4.53
M-MC     1.46
M-LC     1.24
L-LC     1.14
H-LC     1.06
L-SC     0.78
L-MC     0.39
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.30,-9.45,45.36
IT,12.84,-19.70,79.49
FINANCE,11.13,-15.12,63.45
MISC,7.07,-32.39,86.55
ELECTRICAL,5.89,-14.50,56.44
PAINTS,5.79,-14.72,31.76
INSURANCE,4.69,-3.91,39.47
PHARMA,3.92,-7.69,41.59
BANKS,2.80,-37.34,120.56


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3324711.0,21
AR,1403417.0,10
XR,1261587.0,12
X40,1148784.0,15
X40N,952754.0,9
OX40N,724013.0,10
XY25,428409.0,7
SR,289479.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3623228.0,24
X-MC,1657036.0,16
M-SC,1515612.0,15
X-LC,971058.0,13
X-SC,819738.0,8
H-MC,447126.0,3
L-SC,182454.0,2
M-LC,120064.0,1
H-LC,85938.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1269814.0      6
           AR         943893.0      5
M-SC       XY24       866094.0      6
H-SC       XR         811436.0      7
X-MC       X40        545834.0      7
           XY24       462978.0      3
           X40N       419529.0      4
X-LC       X40        409069.0      6
X-SC       X40N       323246.0      3
M-SC       OX40N      322617.0      5
H-SC       OX40N      308606.0      4
X-SC       XY24       302611.0      3
H-SC       SR         289479.0      2
H-MC       AR         238512.0      2
X-MC       XY25       228695.0      2
X-LC       XY24       214600.0      2
           X40N       209979.0      2
H-MC       XY24       208614.0      1
X-SC       X40        193881.0      2
M-SC       XR         191827.0      2
X-LC       XY25       137410.0      3
M-SC       AR         135074.0      2
M-LC       XR         120064.0      1
L-SC       OX40N       92790.0      1
           XR          89664.0      1
H-LC       AR          85938.0      1
L-MC       XR          48596.0      1
M-MC       XY25        42103.0      1
L-LC       XY25        20201.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 38.0 seconds
